# Processing

date: 15.09.2023

# Setup

Activate `npx_10m_384ch_unit_classes` venv

In [1]:
 # listen to changes
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np
import scipy
import spikeinterface.extractors as se 
import os
import spikeinterface as si
import spikeinterface.preprocessing as spre
import spikeinterface.widgets as sw
import spikeinterface.extractors as se 
import shutil
import probeinterface as pi

# SET PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/bernstein_2023/"
os.chdir(PROJ_PATH)

from src.nodes.utils import get_config
from src.nodes.dataeng.silico import recording, probe_wiring
from src.nodes.prepro import preprocess

# SETUP PARAMETERS
EXPERIMENT = "vivo_reyes" 
SIMULATION_DATE = "2015"   
data_conf, param_conf = get_config(EXPERIMENT, SIMULATION_DATE).values()
SAMPFREQ = param_conf["run"]["sampling_frequency"]
SPONTANEOUS_PERIOD = param_conf["S001E009F002_Raw"]["spontaneous_period"]
N_CONTACTS = param_conf["S001E009F002_Raw"]["probe"]["n_contacts"]
GOOD_CHANNELS = param_conf["S001E009F002_Raw"]["good_channels"]

2023-09-15 13:00:40,818 - root - utils.py - get_config - INFO - Reading experiment config.
2023-09-15 13:00:40,826 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Load raw

In [219]:
Recording = se.read_mcsraw(data_conf["raw"])

### Describe

In [220]:
# list properties
Recording._properties.keys()

dict_keys(['gain_to_uV', 'offset_to_uV', 'channel_name'])

In [221]:
gain_to_uV = Recording.get_property('gain_to_uV')
gain_to_uV

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125,
       0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [222]:
offset_to_uV = Recording.get_property('offset_to_uV')
offset_to_uV

array([-4096., -4096., -4096., -4096., -4096., -4096., -4096., -4096.,
       -4096., -4096., -4096., -4096., -4096., -4096., -4096., -4096.,
       -4096., -4096.])

In [223]:
channel_name = Recording.get_property('channel_name')
channel_name

array(['El_01', 'El_02', 'El_03', 'El_04', 'El_05', 'El_06', 'El_07',
       'El_08', 'El_09', 'El_10', 'El_11', 'El_12', 'El_13', 'El_14',
       'El_15', 'El_16', 'El_48', 'El_255'], dtype='<U64')

In [224]:
channel_ids = Recording.get_channel_ids()
channel_ids

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17'], dtype='<U64')

### Cast as RecordingExtractor

In [225]:
# raw_recording.save(folder=data_conf["recording"]["output"], format="binary")

### Curate RecordingExtractor

1. Drop bad channels

In [226]:
# load RecordingExtractor
# Recording = si.load_extractor(data_conf["recording"]["output"])
traces = Recording.get_traces()
traces = traces[:, GOOD_CHANNELS]

# update properties
channel_ids = np.array([str(ci) for ci in GOOD_CHANNELS])
gain_to_uV = gain_to_uV[GOOD_CHANNELS]
offset_to_uV = offset_to_uV[GOOD_CHANNELS]
channel_name = channel_name[GOOD_CHANNELS]

1. Keep spontaneous period only

try simplifying with `Recording = Recording.frame_slice(start_frame=1200*fs, end_frame=1500*fs)`

In [227]:
# get sample times, locate spontaneous timepoints
times = Recording.get_times()
spontaneous_loc = (times >= SPONTANEOUS_PERIOD[0]) & (times < SPONTANEOUS_PERIOD[1])
traces = traces[spontaneous_loc,:]

# create spontaneous recording
Recording = se.NumpyRecording([traces], sampling_frequency=SAMPFREQ, channel_ids=channel_ids)

# reset times
new_times = np.arange(0, SPONTANEOUS_PERIOD[1] - SPONTANEOUS_PERIOD[0], 1/SAMPFREQ)
Recording.set_times(new_times)

/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/2_bernstein_2023/envs/npx_10m_384ch_unit_classes/lib/python3.9/site-packages/spikeinterface/core/baserecording.py:194: UserWarning: Setting times with Recording.set_times() is not recommended because times are not always propagated to across preprocessingUse use this carefully!
  warn('Setting times with Recording.set_times() is not recommended because '


In [228]:
# unit-test spontaneous period duration
assert sum(spontaneous_loc) / SAMPFREQ == SPONTANEOUS_PERIOD[1] - SPONTANEOUS_PERIOD[0], "the period is wrong"

3. Reset properties

In [229]:
Recording.set_property('gain_to_uV', values=gain_to_uV) 
Recording.set_property('offset_to_uV', values=offset_to_uV)
Recording.set_property('channel_name', values=channel_name)

4. Rewrite

In [230]:
# rewrite
shutil.rmtree(data_conf["recording"]["output"], ignore_errors=True)
Recording.save(folder=data_conf["recording"]["output"], format="binary")

write_binary_recording with n_jobs = 1 and chunk_size = None


BinaryFolderRecording: 16 channels - 1 segments - 20.0kHz - 3300.000s

In [232]:
data_conf["recording"]["output"]

'/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/2_bernstein_2023/dataeng/1_vivo/reyes/campaign/recording/traces'

### Probe wiring

see 1

In [155]:
# mapping = np.flip(np.array([27, 26, 28, 25, 31, 22, 30, 23, 33, 20, 34, 19, 32, 21, 29, 24]))
# mapping = mapping - min(mapping)
# probe = pi.generator.generate_linear_probe(num_elec=16, ypitch=100, contact_shapes='circle', contact_shape_params={'radius': 12.5})
# probe.create_auto_shape(probe_type='tip', margin = 50)
# probe.set_device_channel_indices(mapping)
# # pi.plotting.plot_probe(probe, with_channel_index=True, with_device_index=True) 
# Recording.set_probe(probe, in_place = True)

NumpyRecording: 16 channels - 1 segments - 20.0kHz - 3300.000s

### Check Probe

In [4]:
# takes 15 min (5min rec)

# run 
Recording = probe_wiring.run(data_conf, param_conf)

# write 
probe_wiring.write(Recording, data_conf)

2023-09-15 13:16:51,197 - root - probe_wiring.py - run - INFO - probe: vivo_reyes
write_binary_recording with n_jobs = 1 and chunk_size = None
2023-09-15 13:19:12,101 - root - probe_wiring.py - write - INFO - Probe wiring done in  140.4 secs


In [242]:
# check is probe
try: 
    Recording.get_probe() 
except: 
    print("there is no probe wired")

### Preprocess recording

In [5]:
# takes 4 mins

# preprocess (8 min)
Preprocessed = preprocess.run(data_conf, param_conf)

# write
preprocess.write(Preprocessed, data_conf)

# sanity check is preprocessed
print(Preprocessed.is_filtered())

write_binary_recording with n_jobs = 1 and chunk_size = None
True


# Reference

(1) https://bbpgitlab.epfl.ch/conn/personal/imbeni/spike-sorting/-/blob/dev/legacy%20exploratory%20scripts/Reyes_16offline_spont.ipynb